In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [3]:
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 96.8MB/s]


In [4]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [5]:
!pip install -q transformers[torch] accelerate -U
!pip install -q sentence_transformers

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 33.0 MB/s eta 0:00:00
   ━━━━

In [6]:
import numpy as np

import pandas as pd

import os

from tqdm.notebook import tqdm

from tqdm import tqdm

from transformers import BitsAndBytesConfig

from torch.utils.data import DataLoader,Dataset

import torch

import torch.nn as nn

import torch.nn.functional as F



In [334]:
MAX_LENGTH = 512
model_name="intfloat/multilingual-e5-large-instruct"
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["esp"]
# langs=["eng"]
# langs=["afr"]

# label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness']


In [335]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)



In [336]:
train_data=all_csv
train_data=train_data.fillna(0)
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')
train_data = train_data[train_data['clean_message'].isnull()==False]

train_data.reset_index()

In [341]:
train_data['lang'].value_counts()

,count
lang,
esp,1996


In [342]:
sum(train_data['lang'].isna())

0

In [343]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message


In [344]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [345]:
maxi=train_data['clean_message'].apply(len).max()
maxi

440

In [346]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
271,esp_train_track_a_00272,0,0,0,0,1,0,esp,me puse a llorar como si no ubiera un mañana y...


In [347]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['me puse a llorar como si no ubiera un mañana y toda mi familia se rieron de mi crying face smiling face with hearteyes kissing face with smiling eyes face blowing a kiss kissing face with closed eyes smiling cat with hearteyes couple with heart red heart blue heart green heart yellow heart purple heart beating heart broken heart two hearts sparkling heart growing heart heart with arrow heart with ribbon revolving hearts heart decoration'],
      dtype=object)

In [348]:
# shuffle data
train_data=train_data.sample(frac=1)

In [350]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [351]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [352]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        1504      492  1996
disgust      1342      654  1996
fear         1679      317  1996
joy          1354      642  1996
sadness      1687      309  1996
surprise     1575      421  1996


In [353]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [354]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([2.0285, 1.5260, 3.1483, 1.5545, 3.2298, 2.3705], device='cuda:0')

In [356]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [361]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [362]:


torch.cuda.empty_cache()
print(model_name)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        model_name,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

intfloat/multilingual-e5-large-instruct


In [364]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

In [365]:
device="cuda" if torch.cuda.is_available() else 'cpu'
device

In [367]:
embedding_model = embedding_model.to(device)
for param in embedding_model.parameters():
    param.data = param.data.contiguous()

In [368]:
# Custom Dataset for batching
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [370]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [371]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 63/63 [00:30<00:00,  2.07it/s]


In [372]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [373]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [374]:
import xgboost as xgb

In [375]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [376]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [377]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [378]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [379]:
!rm -rf svm*

In [380]:
from sklearn.svm import SVC

In [381]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: disgust
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [382]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [383]:
!zip svm_models_pkl.zip svm_model_*.pkl

  adding: svm_model_anger.pkl (deflated 75%)
  adding: svm_model_disgust.pkl (deflated 75%)
  adding: svm_model_fear.pkl (deflated 75%)
  adding: svm_model_joy.pkl (deflated 75%)
  adding: svm_model_sadness.pkl (deflated 75%)
  adding: svm_model_surprise.pkl (deflated 75%)


In [384]:
kind="dev"

In [385]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


,index,id,anger,disgust,fear,joy,sadness,surprise,lang,clean_message
0,0,esp_dev_track_a_00001,0,0,0,0,0,0,esp,lástima que este roast va a desaparecer
1,1,esp_dev_track_a_00002,0,0,0,0,0,0,esp,la verdad 3lla tiene razón los trajes son horr...
2,2,esp_dev_track_a_00003,0,0,0,0,0,0,esp,mi padre me rompio el corazon broken heart nun...
3,3,esp_dev_track_a_00004,0,0,0,0,0,0,esp,deberás sus vídeos están super horribles y sus...
4,4,esp_dev_track_a_00005,0,0,0,0,0,0,esp,que asco alto cringe larguense de youtube esto...
...,...,...,...,...,...,...,...,...,...,...
179,179,esp_dev_track_a_00180,0,0,0,0,0,0,esp,sinceramente los vídeos como estos me hacen ll...
180,180,esp_dev_track_a_00181,0,0,0,0,0,0,esp,crying face que injusticia urge permitir que l...
181,181,esp_dev_track_a_00182,0,0,0,0,0,0,esp,llore pensar que yo te seguí desde tu primer v...
182,182,esp_dev_track_a_00183,0,0,0,0,0,0,esp,no entendí xd están o no


In [386]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [387]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [388]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 6/6 [00:03<00:00,  1.72it/s]


In [389]:
y_dev_flat = np.array(y_dev)

In [390]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: disgust
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [391]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [392]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [393]:
!rm -rf track*

In [394]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'esp' saved to track_a/pred_esp.csv


In [395]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/pred_esp.csv (deflated 89%)
